In [104]:
import numpy as np
import pickle

import numpy as np
import torch
import torch.nn as nn

### Model 1 - Linear Regression ###

In [105]:
#load train test and validation data from tensor_collection
cm_x_train = torch.load('tensor_collection\\cm_x_train.pt')
cm_y_train = torch.load('tensor_collection\\cm_y_train.pt')
cm_x_test = torch.load('tensor_collection\\cm_x_test.pt')
cm_y_test = torch.load('tensor_collection\\cm_y_test.pt')
cm_x_val = torch.load('tensor_collection\\cm_x_val.pt')
cm_y_val = torch.load('tensor_collection\\cm_y_val.pt')



In [106]:
#Loss function
def mean_squared_error(x : np.ndarray, y : np.ndarray, theta : np.ndarray) -> np.ndarray:
    yhat = x @ theta 
    error = yhat - y 
    loss = (1 / len(y)) * np.sum(error ** 2) 
    return loss

#Gradient descent
def calculate_gradient_and_update(x: np.ndarray, y: np.ndarray, theta: np.ndarray, alpha: float) -> tuple([float, np.ndarray]):
    gradient = (1 / len(y)) * x.T @ ((x @ theta) - y) 
    theta_new = theta - (alpha * gradient) 
    loss = mean_squared_error(x, y, theta_new)
    return loss, theta_new


In [107]:
num_features = cm_x_train.shape[1]

train_loss_history = []
val_loss_history = []
num_epochs = 10000
alpha = 0.1

# Initialize theta to random values between -2 and 2
theta = np.random.uniform(-2, 2, (num_features))

# Training Loop
for t in range(num_epochs):
    train_loss, theta = calculate_gradient_and_update(cm_x_train, cm_y_train, theta, alpha)
    train_loss_history.append(train_loss)

    # Validation step
    val_loss, _ = calculate_gradient_and_update(cm_x_val, cm_y_val, theta, 0)  # alpha=0 to prevent updates
    val_loss_history.append(val_loss)


# Plot loss history
print('Final training loss:', train_loss)
print('Final validation loss:', val_loss)


Final training loss: 66.8928832558388
Final validation loss: 63.11712690253442


In [108]:
#save theta and loss history for analysis notebook
pickle.dump(theta, open('tensor_collection\\regression_theta.pkl', 'wb'))
pickle.dump(train_loss, open('tensor_collection\\regression_train_loss.pkl', 'wb'))
pickle.dump(val_loss, open('tensor_collection\\regression_val_loss.pkl', 'wb'))

In [109]:
cuda_available = torch.cuda.is_available()
print("Is CUDA available? ", cuda_available)

Is CUDA available?  True


In [110]:
#load torch tensors
cnn_x_train_tensor = torch.load('tensor_collection\\cnn_x_train.pt')
cnn_x_val_tensor = torch.load('tensor_collection\\cnn_x_val.pt')
cnn_y_train_tensor = torch.load('tensor_collection\\cnn_y_train.pt')
cnn_y_val_tensor = torch.load('tensor_collection\\cnn_y_val.pt')

In [111]:
if (torch.cuda.is_available()):
    device = torch.device("cuda:0")
    print('Using GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('GPU not available, using CPU')
    

Using GPU: NVIDIA GeForce GTX 1660 Ti


In [129]:
from dataloader import CustomDataloader
from networks import CNNRegression
import tqdm

print("Initial image tensor shape:", cnn_x_train_tensor.shape)
#cm_x_train, cm_x_val
train_dataloader = CustomDataloader(cnn_x_train_tensor, cnn_y_train_tensor.unsqueeze(1), batch_size=50, randomize=True)
val_dataloader = CustomDataloader(cnn_x_val_tensor, cnn_y_val_tensor.unsqueeze(1), batch_size=50, randomize=False)

model = CNNRegression().to(device)

# instantiate your optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# log your losses
train_losses = []
val_losses = []

# define how many epochs to train on
epochs = 200


#print contents of batch


# define your loss function for multiclass classification task
# BCE does binary cross entropy automatically for each class
loss_fn = torch.nn.MSELoss()
for _ in tqdm.tqdm(range(epochs)):
    losses = []
    for _ in range(train_dataloader.num_batches_per_epoch):
        train_batch = train_dataloader.fetch_batch()
        x_batch = train_batch['x_batch'].to(device)
        y_batch = train_batch['y_batch'].to(device)
        yhat = model(x_batch)  # Use only image batch
        train_loss = torch.mean(loss_fn(yhat, y_batch))  # Adjusted to label_batch

        # Training data backward pass
        train_loss.backward()
        optimizer.step()
        losses.append(train_loss.detach().cpu().numpy())

    train_losses.extend(losses)
    
    losses = []
    for _ in range(val_dataloader.num_batches_per_epoch):
        # Validation data forward pass only
        val_batch = val_dataloader.fetch_batch()
        x_batch = val_batch['x_batch'].to(device)
        y_batch = val_batch['y_batch'].to(device)
        yhat = model(x_batch)  # Use only image batch
        val_loss = torch.mean(loss_fn(yhat, y_batch), axis=0)  # Adjusted to label_batch
        losses.append(val_loss.detach().cpu().numpy())

    val_losses.append(np.mean(losses))


Initial image tensor shape: torch.Size([18966, 1, 64, 64])


100%|██████████| 200/200 [04:23<00:00,  1.32s/it]


In [131]:
#store train and validation losses
torch.save(train_losses, 'tensor_collection\\cnn_train_losses.pt')
torch.save(val_losses, 'tensor_collection\\cnn_val_losses.pt')
#store epochs for plotting
torch.save(epochs, 'tensor_collection\\cnn_epochs.pt')

